In [1]:
import torch
import torchvision  # To be able to access standard datasets more easily
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import numpy as np  # To plot and display stuff
import torch.optim as optim  # Where the optimization modules are
import urllib.request
from random import randint
from mlxtend.data import loadlocal_mnist
import platform

#### Exercise 1.

**Modify the code of mnist-assignment.py to train the chosen network using SGD with minibatches of different sizes (starting from size 1). Discuss how the SGD method with various
minibatch sizes compares in terms of speed of convergence and final accuracy on test set with
GD.**

In [2]:
# Using torchvision we can conveniently load some datasets
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=ToTensor())
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=ToTensor())

# Extract tensor of data and labels for both the training and the test set
x, y = trainset.data.float(), trainset.targets
x_test, y_test = testset.data.float(), testset.targets


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [9]:
trainset.data.shape

torch.Size([60000, 28, 28])

In [10]:
x.shape[0]

60000

In [12]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

train_ds = TensorDataset(x, y)
valid_ds = TensorDataset(x_test, y_test)

image_size = trainset.data.shape[1] * trainset.data.shape[2]

#net = torch.nn.Sequential(
#    torch.nn.Linear(image_size, 300),
#    torch.nn.Sigmoid(),
#    torch.nn.Linear(300, 10),
#)

In [ ]:
%%time

n_epoch = 23
train_risks = []
test_risks = []
n_neurons = 300
batch_sizes = [1, 3000, 6000, 12000, x.shape[0]]
model = torch.nn.Sequential(
    torch.nn.Linear(image_size, n_neurons),
    torch.nn.Sigmoid(),
    torch.nn.Linear(n_neurons, n_neurons),
    torch.nn.Sigmoid(),
    torch.nn.Linear(n_neurons, n_neurons),
    torch.nn.Sigmoid(),
    torch.nn.Linear(n_neurons, 10),
)
optimizer = optim.SGD(model.parameters(), lr=0.1)
loss_func = F.cross_entropy

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean() * 100

for bs in batch_sizes:
    print(f'bs={bs}')
    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
    valid_dl = DataLoader(valid_ds, batch_size=bs * 2)
    valid_accs_per_epoch = []
    for epoch in range(n_epoch):
        model.train()
        for xb, yb in train_dl:
            optimizer.zero_grad()
            inputs = torch.flatten(xb, start_dim=1, end_dim=2)
            outputs = model(inputs)
            # Define the empirical risk
            Risk = loss_func(outputs, yb)
            # Make the backward step
            Risk.backward()
            # Update the parameters
            optimizer.step()
        model.eval()
        with torch.no_grad():
            valid_accs = []
            for xb, yb in valid_dl:
                valid_input = torch.flatten(xb, start_dim=1, end_dim=2)
                valid_accs.append(accuracy(model(valid_input), yb))
        valid_accs_per_epoch.append(np.mean(valid_accs))
            # test_prediction = net(torch.flatten(x_test, start_dim=1, end_dim=2))
            # test_acc = accuracy(test_prediction, y_test)
            # test_accs.append(test_acc)

    # train_risk = Risk.item()
    # train_risks.append(train_risk)
    #
    # test_prediction = net(torch.flatten(x_test, start_dim=1, end_dim=2))
    # test_risk = loss_func(test_prediction, y_test)
    # test_risks.append(test_risk)
    plt.plot(np.arange(1,n_epoch+1,1), valid_accs_per_epoch, label=f'bs = {bs}')

plt.legend()
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.show()

bs=1


#### Excersices 2
**Initialize all the weights and all biases to a common value (for instance 0). Try to train
the network either with GD or SGD. What happens? Why?**

In [39]:
print(x)
print(x[0].shape)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [44]:
from sklearn.datasets import load_digits

x, y = load_digits(return_X_y=True)

n_samples, n_features = x.shape
n_classes = len(np.unique(y))

weights = torch.randn(n_features, n_classes) / np.sqrt(n_features)
bias = torch.randn(n_classes) / np.sqrt(n_features)
weights.requires_grad_()
bias.requires_grad_()


model = torch.nn.Sequential(
    torch.nn.Linear(image_size, n_neurons),
    torch.nn.Sigmoid(),
    torch.nn.Linear(n_neurons, 10),
)

model[0].weight.shape

torch.Size([300, 784])

The smaller the minibatches, the faster it will converge